#### Hyperparameter Tuning

In [9]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

#### Dataset creation

In [10]:
# Create a DataFrame for classification containing four real features and one binary target

df = pd.DataFrame({
    'feature1': np.random.randint(0, 100, 100),
    'feature2': np.random.randint(0, 100, 100),
    'feature3': np.random.randint(0, 100, 100),
    'feature4': np.random.randint(0, 100, 100),
    'target': np.random.randint(0, 2, 100)
})

In [11]:
df.head()

,feature1,feature2,feature3,feature4,target
0,71,5,95,37,0
1,95,51,61,19,0
2,30,86,55,13,1
3,4,49,56,24,1
4,21,88,30,32,1


In [12]:
train_df = df[:50]
validation_df = df[50:80]

In [13]:
dt = DecisionTreeClassifier()
dt.fit(train_df[['feature1', 'feature2', 'feature3', 'feature4']], train_df['target'])
dt

DecisionTreeClassifier()

In [14]:
dt.score(validation_df[['feature1', 'feature2', 'feature3', 'feature4']], validation_df['target'])

0.6

In [15]:
dt = DecisionTreeClassifier(criterion='entropy', max_depth=2)
dt.fit(train_df[['feature1', 'feature2', 'feature3', 'feature4']], train_df['target'])
dt.score(validation_df[['feature1', 'feature2', 'feature3', 'feature4']], validation_df['target'])


0.4

In [16]:
hyperparams = {'criterion': ['gini', 'entropy'],
               'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
               'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]}


In [17]:
out = {}
for c in hyperparams['criterion']:
    for d in hyperparams['max_depth']:
        for s in hyperparams['min_samples_split']:
            dt = DecisionTreeClassifier(criterion=c, max_depth=d, min_samples_split=s)
            dt.fit(train_df[['feature1', 'feature2', 'feature3', 'feature4']], train_df['target'])
            out[(c, d, s)] = dt.score(validation_df[['feature1', 'feature2', 'feature3', 'feature4']], validation_df['target'])

In [18]:
hp_ser = pd.Series(out)
hp_ser.sort_values(ascending=False)

gini     9   6     0.666667
         7   6     0.666667
         8   6     0.666667
         10  6     0.666667
         6   6     0.666667
                     ...   
entropy  2   10    0.400000
             5     0.400000
             4     0.400000
             3     0.400000
             2     0.400000
Length: 162, dtype: float64

In [19]:
hp_ser.idxmax()

('gini', 6, 6)

#### Without using multiple nested loops

In [20]:
print(hyperparams.items(), len(hyperparams.items()))

dict_items([('criterion', ['gini', 'entropy']), ('max_depth', [2, 3, 4, 5, 6, 7, 8, 9, 10]), ('min_samples_split', [2, 3, 4, 5, 6, 7, 8, 9, 10])]) 3


In [21]:
print_vec(*hyperparams.items())

[('criterion', ['gini', 'entropy']) 
('max_depth', [2, 3, 4, 5, 6, 7, 8, 9, 10]) 
('min_samples_split', [2, 3, 4, 5, 6, 7, 8, 9, 10])]


In [22]:
list(zip(*hyperparams.items()))

[('criterion', 'max_depth', 'min_samples_split'),
 (['gini', 'entropy'],
  [2, 3, 4, 5, 6, 7, 8, 9, 10],
  [2, 3, 4, 5, 6, 7, 8, 9, 10])]

In [23]:
keys, values = zip(*hyperparams.items())

In [24]:
keys

('criterion', 'max_depth', 'min_samples_split')

In [25]:
values

(['gini', 'entropy'],
 [2, 3, 4, 5, 6, 7, 8, 9, 10],
 [2, 3, 4, 5, 6, 7, 8, 9, 10])

In [26]:
list(itertools.product(*values))[::10]

[('gini', 2, 2),
 ('gini', 3, 3),
 ('gini', 4, 4),
 ('gini', 5, 5),
 ('gini', 6, 6),
 ('gini', 7, 7),
 ('gini', 8, 8),
 ('gini', 9, 9),
 ('gini', 10, 10),
 ('entropy', 3, 2),
 ('entropy', 4, 3),
 ('entropy', 5, 4),
 ('entropy', 6, 5),
 ('entropy', 7, 6),
 ('entropy', 8, 7),
 ('entropy', 9, 8),
 ('entropy', 10, 9)]

In [27]:
v = next(itertools.product(*values))
print(v)

('gini', 2, 2)


In [28]:
print_vec(*zip(keys, v))

[('criterion', 'gini') 
('max_depth', 2) 
('min_samples_split', 2)]


In [29]:
def print_dict(**kwargs):
    print(kwargs)

print_dict(**(dict(zip(keys, v))))

{'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 2}


In [30]:
out = {}
for v in itertools.product(*values):
    params = (dict(zip(keys, v)))
    dt= DecisionTreeClassifier(**params)
    dt.fit(train_df[['feature1', 'feature2', 'feature3', 'feature4']], train_df['target'])
    out[(params['criterion'], params['max_depth'], params['min_samples_split'])] = dt.score(validation_df[['feature1', 'feature2', 'feature3', 'feature4']], validation_df['target'])
    

In [31]:
pd.Series(out).sort_values(ascending=False)

gini     9   6     0.666667
         7   6     0.666667
         8   6     0.666667
         10  6     0.666667
         6   6     0.666667
                     ...   
entropy  2   10    0.400000
             7     0.400000
             8     0.400000
             9     0.400000
             2     0.400000
Length: 162, dtype: float64